In [2]:
import pandas as pd
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
X=df.drop('Outcome',axis=1).values
y=df['Outcome'].values

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=0)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

### Decoder

In [51]:
class Decoder(nn.Module):
    def __init__(self,input_features=8,hidden1=8,hidden2=200,hidden3=200,hidden4=300,hidden5=300,hidden6=400,hidden7=400,hidden8=300,hidden9=300,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.f_connected3=nn.Linear(hidden2,hidden3)
        self.f_connected4=nn.Linear(hidden3,hidden4)
        self.f_connected5=nn.Linear(hidden4,hidden5)
        self.f_connected6=nn.Linear(hidden5,hidden6)
        self.f_connected7=nn.Linear(hidden6,hidden7)
        self.f_connected8=nn.Linear(hidden7,hidden8)
        self.f_connected9=nn.Linear(hidden8,hidden9)
        self.out=nn.Linear(hidden9,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=F.relu(self.f_connected3(x))
        x=F.relu(self.f_connected4(x))
        x=F.relu(self.f_connected5(x))
        x=F.relu(self.f_connected6(x))
        x=F.relu(self.f_connected7(x))
        x=F.relu(self.f_connected8(x))
        x=F.relu(self.f_connected9(x))
        x=self.out(x)
        return x

In [52]:
torch.manual_seed(20)
model = Decoder()

In [53]:
model.parameters

<bound method Module.parameters of Decoder(
  (f_connected1): Linear(in_features=8, out_features=8, bias=True)
  (f_connected2): Linear(in_features=8, out_features=200, bias=True)
  (f_connected3): Linear(in_features=200, out_features=200, bias=True)
  (f_connected4): Linear(in_features=200, out_features=300, bias=True)
  (f_connected5): Linear(in_features=300, out_features=300, bias=True)
  (f_connected6): Linear(in_features=300, out_features=400, bias=True)
  (f_connected7): Linear(in_features=400, out_features=400, bias=True)
  (f_connected8): Linear(in_features=400, out_features=300, bias=True)
  (f_connected9): Linear(in_features=300, out_features=300, bias=True)
  (out): Linear(in_features=300, out_features=2, bias=True)
)>

In [54]:
model.parameters()

<generator object Module.parameters at 0x7fc4b8c18820>

In [55]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01) 

In [56]:
epochs = 500
final_losses = []
for i in range(epochs):
    i = i + 1
    y_pred = model.forward(X_train)
    loss=loss_function(y_pred, y_train)
    final_losses.append(loss.item())
    if i%10==1:
        print("Epoch number: {} and the loss: {}".format(i, loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch number: 1 and the loss: 0.6994479298591614
Epoch number: 11 and the loss: 0.6807885766029358
Epoch number: 21 and the loss: 0.6516156196594238
Epoch number: 31 and the loss: 0.6306445002555847
Epoch number: 41 and the loss: 0.6166210174560547
Epoch number: 51 and the loss: 0.5757931470870972
Epoch number: 61 and the loss: 0.5477321147918701
Epoch number: 71 and the loss: 0.5501350164413452
Epoch number: 81 and the loss: 0.541769802570343
Epoch number: 91 and the loss: 0.5239008069038391
Epoch number: 101 and the loss: 0.5185975432395935
Epoch number: 111 and the loss: 0.526075005531311
Epoch number: 121 and the loss: 0.5419977307319641
Epoch number: 131 and the loss: 0.508708655834198
Epoch number: 141 and the loss: 0.49814125895500183
Epoch number: 151 and the loss: 0.4873463213443756
Epoch number: 161 and the loss: 0.47971105575561523
Epoch number: 171 and the loss: 0.4749397337436676
Epoch number: 181 and the loss: 0.4853869676589966
Epoch number: 191 and the loss: 0.488510996

### Classifier

In [9]:
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

In [27]:
class Classifier(nn.Module):
    def __init__(self,input_features=8,hidden1=8,hidden2=200,hidden3=200,hidden4=300,hidden5=300,hidden6=400,hidden7=400, out_features=1):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        #self.f_connected3=nn.Linear(hidden2,hidden3)
        self.f_connected4=nn.Linear(hidden3,hidden4)
        #self.f_connected5=nn.Linear(hidden4,hidden5)
        self.f_connected6=nn.Linear(hidden5,hidden6)
        self.f_connected7=nn.Linear(hidden6,hidden7)
        #reshape using tensor_name.reshape(20,20,1)
        self.conv1 = torch.nn.Conv2d(1, 100, kernel_size = (2,6), stride = 1)
        self.f_connected8=nn.Linear(1800,35)
        self.out=nn.Linear(35,out_features)
    def forward(self,x):
        dropout=nn.Dropout(p=0.2)
        pooling=nn.MaxPool2d((2,6))
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        dropout(x)
        #x=F.relu(self.f_connected3(x))
        x=F.relu(self.f_connected4(x))
        dropout(x)
        #x=F.relu(self.f_connected5(x))
        x=F.relu(self.f_connected6(x))
        x=F.relu(self.f_connected7(x))
        #print(x.shape)
        x=x.view(-1,1,20,20)
        x=F.relu(self.conv1(x))
        x = dropout(x)
        x = pooling(x)
        x = dropout(x)
        x=x.view(-1, 9*2*100)
        #x = torch.flatten(x)
        #x.reshape(-1)
        #x.flatten()
        #x.reshape([1,9*2*100])
        x=F.relu(self.f_connected8(x))
        x=F.sigmoid(self.out(x))
        return x

In [28]:
torch.manual_seed(20)
model_classifier = Classifier()

In [29]:
model_classifier.parameters

<bound method Module.parameters of Classifier(
  (f_connected1): Linear(in_features=8, out_features=8, bias=True)
  (f_connected2): Linear(in_features=8, out_features=200, bias=True)
  (f_connected4): Linear(in_features=200, out_features=300, bias=True)
  (f_connected6): Linear(in_features=300, out_features=400, bias=True)
  (f_connected7): Linear(in_features=400, out_features=400, bias=True)
  (conv1): Conv2d(1, 100, kernel_size=(2, 6), stride=(1, 1))
  (f_connected8): Linear(in_features=1800, out_features=35, bias=True)
  (out): Linear(in_features=35, out_features=1, bias=True)
)>

In [46]:
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model_classifier.parameters(), lr = 0.01) 

In [47]:
epochs = 600
final_losses_classifier = []
for i in range(epochs):
    i = i + 1
    y_pred = model_classifier.forward(X_train)
    loss=loss_function(y_pred, y_train)
    final_losses_classifier.append(loss.item())
    if i%10==1:
        print("Epoch number: {} and the loss: {}".format(i, loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

ValueError: Using a target size (torch.Size([691])) that is different to the input size (torch.Size([691, 1])) is deprecated. Please ensure they have the same size.

#### Check:
In the nn made in classfier, the output layer has 2 neurons instead of 1 as given in the paper
